# Sample code how to use `logllm` package

## Your machine learning code is here

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


iris = datasets.load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = SVC(kernel='linear')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")

## Start logging with `logllm`

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from logllm import logllm

notebook_path = "svc-sample.ipynb" # Here is target file to log
project_name = "logllm-sample" # project name for wandb

logllm(notebook_path,project_name)